In [58]:
import pandas as pd
import math
import numpy as np
import datetime 


In [59]:
dataset = pd.read_csv('./datasets/dataset_hourly.csv')


dataset = dataset.drop(columns=['Unnamed: 0'])
dataset = dataset.rename(columns={"block_time":"timestamp"})
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset['timestamp'] = dataset['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625
...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688


In [60]:
dataset_tip = pd.read_csv('./datasets/dataset_threshold.csv')
dataset_tip = dataset_tip.drop(columns=['Unnamed: 0'])
dataset_tip = dataset_tip.rename(columns={"block_time":"timestamp"})
dataset_tip['timestamp'] = pd.to_datetime(dataset_tip['timestamp'])
dataset_tip['timestamp'] = dataset_tip['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
dataset_tip['timestamp'] = pd.to_datetime(dataset_tip['timestamp'])

dataset = dataset.merge(dataset_tip, on='timestamp', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])

dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976,0.296145,0.165969,0.426321
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797,0.318663,0.185114,0.476079
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939,0.505857,0.234593,0.677423
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827,0.221217,0.104981,0.403270
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625,0.391439,0.179006,0.498325
...,...,...,...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513,0.184175,0.069670,0.337023
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094,0.162364,0.110703,0.278258
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906,0.156575,0.103242,0.309302
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688,0.236753,0.119655,0.454288


In [61]:
# eth price dataset from Binance
eth_price = pd.read_csv('./datasets/eth_price.csv')
eth_price['datetime'] = pd.to_datetime(eth_price['datetime'])
eth_price = eth_price[eth_price['datetime'] <= '2022-08-05 12:33:00']
eth_price = eth_price.rename(columns={'datetime': 'timestamp'})
eth_price = eth_price.drop(columns=['Unnamed: 0'])
eth_price = eth_price.drop_duplicates(subset=['timestamp'])
eth_price

,timestamp,eth_price
0,2021-08-05 11:00:00,2622.22
1,2021-08-05 12:00:00,2606.80
2,2021-08-05 13:00:00,2686.49
3,2021-08-05 14:00:00,2802.95
4,2021-08-05 15:00:00,2755.08
...,...,...
8763,2022-08-05 08:00:00,1657.28
8764,2022-08-05 09:00:00,1661.87
8765,2022-08-05 10:00:00,1669.11
8766,2022-08-05 11:00:00,1713.43


In [62]:
dataset = dataset.merge(eth_price, on='timestamp', how='left', indicator=True)

dataset = dataset.drop(columns=['_merge'])
dataset['date'] = pd.to_datetime(dataset['timestamp'].dt.date) - datetime.timedelta(days=1)
dataset = dataset.dropna()
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip,eth_price,date
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976,0.296145,0.165969,0.426321,2606.80,2021-08-04
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797,0.318663,0.185114,0.476079,2686.49,2021-08-04
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939,0.505857,0.234593,0.677423,2802.95,2021-08-04
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827,0.221217,0.104981,0.403270,2755.08,2021-08-04
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625,0.391439,0.179006,0.498325,2787.96,2021-08-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513,0.184175,0.069670,0.337023,1657.28,2022-08-04
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094,0.162364,0.110703,0.278258,1661.87,2022-08-04
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906,0.156575,0.103242,0.309302,1669.11,2022-08-04
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688,0.236753,0.119655,0.454288,1713.43,2022-08-04


In [63]:
# Fear and Green Index
fag = pd.read_csv('./datasets/fear_and_greed_index.csv')
fag['date'] = pd.to_datetime(fag['date'], format='%d-%m-%Y')

dataset = dataset.merge(fag, on='date', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip,eth_price,date,fng_value,fng_classification
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976,0.296145,0.165969,0.426321,2606.80,2021-08-04,42,Fear
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797,0.318663,0.185114,0.476079,2686.49,2021-08-04,42,Fear
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939,0.505857,0.234593,0.677423,2802.95,2021-08-04,42,Fear
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827,0.221217,0.104981,0.403270,2755.08,2021-08-04,42,Fear
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625,0.391439,0.179006,0.498325,2787.96,2021-08-04,42,Fear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513,0.184175,0.069670,0.337023,1657.28,2022-08-04,30,Fear
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094,0.162364,0.110703,0.278258,1661.87,2022-08-04,30,Fear
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906,0.156575,0.103242,0.309302,1669.11,2022-08-04,30,Fear
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688,0.236753,0.119655,0.454288,1713.43,2022-08-04,30,Fear


In [64]:
# TVL dataset from Defillama
tvl = pd.read_csv('./datasets/defillama_tvl.csv')
tvl['date'] = pd.to_datetime(tvl['date'], format='%d/%m/%Y')
tvl = tvl.drop(columns=['Unnamed: 0'])

dataset = dataset.merge(tvl, on='date', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,q3_tip,eth_price,date,fng_value,fng_classification,ethereum_tvl,polygon_tvl,bsc_tvl
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976,0.296145,0.165969,0.426321,2606.80,2021-08-04,42,Fear,6.589599e+10,5.755411e+09,1.044607e+10
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797,0.318663,0.185114,0.476079,2686.49,2021-08-04,42,Fear,6.589599e+10,5.755411e+09,1.044607e+10
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939,0.505857,0.234593,0.677423,2802.95,2021-08-04,42,Fear,6.589599e+10,5.755411e+09,1.044607e+10
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827,0.221217,0.104981,0.403270,2755.08,2021-08-04,42,Fear,6.589599e+10,5.755411e+09,1.044607e+10
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625,0.391439,0.179006,0.498325,2787.96,2021-08-04,42,Fear,6.589599e+10,5.755411e+09,1.044607e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513,0.184175,0.069670,0.337023,1657.28,2022-08-04,30,Fear,3.634261e+10,1.591160e+09,5.691703e+09
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094,0.162364,0.110703,0.278258,1661.87,2022-08-04,30,Fear,3.634261e+10,1.591160e+09,5.691703e+09
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906,0.156575,0.103242,0.309302,1669.11,2022-08-04,30,Fear,3.634261e+10,1.591160e+09,5.691703e+09
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688,0.236753,0.119655,0.454288,1713.43,2022-08-04,30,Fear,3.634261e+10,1.591160e+09,5.691703e+09


In [65]:
# Polygon dataset
polygon = pd.read_csv('./datasets/polygon.csv')
polygon = polygon.rename(columns={"block_time":"timestamp", "median_tip":"polygon_median_tip", "q1_tip":"polygon_q1_tip", "q3_tip":"polygon_q3_tip", "base_fee":"polygon_base_fee", "blocks":"polygon_blocks"
                                    , "gas_limit":"polygon_gas_limit", "gas_used":"polygon_gas_used", "legacy_tx":"polygon_legacy_tx", "sum_priority_fee_eth":"polygon_sum_priority_fee_eth", "tx":"polygon_tx"})
polygon['timestamp'] = pd.to_datetime(polygon['timestamp'])
polygon['timestamp'] = polygon['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
polygon['timestamp'] = pd.to_datetime(polygon['timestamp'])
polygon = polygon[polygon['timestamp'] <= '2022-08-05 13:00:00']
polygon = polygon[polygon['timestamp'] >= '2021-08-05 12:00:00']
polygon = polygon.drop(columns=['Unnamed: 0'])
polygon = polygon.drop_duplicates(subset=['timestamp'])
polygon = polygon.fillna(0)

# Join dataset and eth price
dataset = dataset.merge(polygon, on='timestamp', how='left', indicator=True)
for c in [s for s in dataset.columns if s in ['polygon_median_tip', 'polygon_q1_tip', 'polygon_q3_tip', 'polygon_base_fee', 'polygon_blocks', 'polygon_gas_limit', 'polygon_gas_used', 'polygon_legacy_tx', 'polygon_legacy_tx', 'polygon_sum_priority_fee_eth', 'polygon_tx']]:
    dataset[c].fillna(0, inplace=True)

dataset = dataset.drop(columns=['_merge'])

dataset


,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,...,polygon_base_fee,polygon_blocks,polygon_gas_limit,polygon_gas_used,polygon_legacy_tx,polygon_sum_priority_fee_eth,polygon_tx,polygon_median_tip,polygon_q1_tip,polygon_q3_tip
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976,0.296145,0.165969,...,0.000000,568.0,1.139787e+10,1.009970e+10,90024.0,0.000000,90024.0,0.0,0.0,0.00000
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797,0.318663,0.185114,...,0.000000,1460.0,2.935993e+10,2.475197e+10,193470.0,0.000000,193470.0,0.0,0.0,0.00000
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939,0.505857,0.234593,...,0.000000,1351.0,2.718620e+10,2.615760e+10,248212.0,0.000000,248212.0,0.0,0.0,0.00000
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827,0.221217,0.104981,...,0.000000,1391.0,2.805965e+10,2.737509e+10,245530.0,0.000000,245530.0,0.0,0.0,0.00000
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625,0.391439,0.179006,...,0.000000,1187.0,2.401471e+10,2.295256e+10,202826.0,0.000000,202826.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513,0.184175,0.069670,...,0.000069,1579.0,4.668661e+10,2.241820e+10,88598.0,1602.557884,137703.0,30.0,30.0,30.00000
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094,0.162364,0.110703,...,0.000179,1483.0,4.405098e+10,2.161803e+10,84626.0,1336.228667,139333.0,30.0,30.0,30.00000
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906,0.156575,0.103242,...,0.000045,1510.0,4.460326e+10,1.969151e+10,71156.0,1549.055448,118805.0,30.0,30.0,30.00000
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688,0.236753,0.119655,...,0.001073,1572.0,4.550043e+10,2.163185e+10,74965.0,3222.291183,128020.0,30.0,30.0,30.00000


In [66]:
# BSC dataset
bsc = pd.read_csv('./datasets/bsc.csv')
bsc = bsc.rename(columns={"block_time":"timestamp","blocks":"bsc_blocks", "gas_limit":"bsc_gas_limit", "gas_used":"bsc_gas_used", "tx":"bsc_tx", 'hourly_median_fee_tx': 'bsc_hourly_median_fee_tx'})
bsc['timestamp'] = pd.to_datetime(bsc['timestamp'])
bsc['timestamp'] = bsc['timestamp'].dt.strftime('%Y-%m-%d %H:%M:00')
bsc['timestamp'] = pd.to_datetime(bsc['timestamp'])
bsc = bsc[bsc['timestamp'] <= '2022-08-05 13:00:00']
bsc = bsc[bsc['timestamp'] >= '2021-08-05 12:00:00']
bsc = bsc.drop(columns=['Unnamed: 0', 'legacy_tx'])
bsc = bsc.drop_duplicates(subset=['timestamp'])

# Join dataset and eth price
dataset = dataset.merge(bsc, on='timestamp', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,...,polygon_sum_priority_fee_eth,polygon_tx,polygon_median_tip,polygon_q1_tip,polygon_q3_tip,bsc_blocks,bsc_gas_limit,bsc_gas_used,bsc_hourly_median_fee_tx,bsc_tx
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976,0.296145,0.165969,...,0.000000,90024.0,0.0,0.0,0.00000,495,37415387991,28170359969,6.998250e+14,193410
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797,0.318663,0.185114,...,0.000000,193470.0,0.0,0.0,0.00000,1149,89525667818,70406168257,6.998400e+14,488676
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939,0.505857,0.234593,...,0.000000,248212.0,0.0,0.0,0.00000,1178,94367832432,72408594411,6.996675e+14,506601
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827,0.221217,0.104981,...,0.000000,245530.0,0.0,0.0,0.00000,1141,92418654982,74179523926,6.990400e+14,527460
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625,0.391439,0.179006,...,0.000000,202826.0,0.0,0.0,0.00000,1168,91988565614,68022847437,6.990675e+14,476592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513,0.184175,0.069670,...,1602.557884,137703.0,30.0,30.0,30.00000,1200,96604011823,20567595097,3.615750e+14,171903
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094,0.162364,0.110703,...,1336.228667,139333.0,30.0,30.0,30.00000,1191,96021398043,20360374766,3.612700e+14,171287
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906,0.156575,0.103242,...,1549.055448,118805.0,30.0,30.0,30.00000,1198,96408427133,19539454407,3.550238e+14,161500
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688,0.236753,0.119655,...,3222.291183,128020.0,30.0,30.0,30.00000,1200,96576987874,21336701829,3.631964e+14,193006


In [67]:
# NFT Data
nft = pd.read_csv('./datasets/nft.csv')
nft = nft.rename(columns={"timestamp":"date"})
nft = nft.drop(columns=['Unnamed: 0'])
nft['date'] = pd.to_datetime(nft['date'], format='%Y-%m-%d')


dataset = dataset.merge(nft, on='date', how='left', indicator=True)
dataset = dataset.drop(columns=['_merge'])
dataset

,base_fee,timestamp,blocks,gas_limit,gas_used,legacy_tx,sum_priority_fee_eth,tx,median_tip,q1_tip,...,mfers_sales_count,deadfellaz_floor_usd,deadfellaz_volume_usd,deadfellaz_sales_count,beeple_floor_usd,beeple_volume_usd,beeple_sales_count,clonex_floor_usd,clonex_volume_usd,clonex_sales_count
0,47.703934,2021-08-05 12:00:00,108,3187808029,2154963245,19619,36.405995,19976,0.296145,0.165969,...,0,0.0000,0.000000,0,0.0000,0.0,0,0.0000,0.00000,0
1,207.615140,2021-08-05 13:00:00,243,6910676231,3722322759,40617,66.263737,41797,0.318663,0.185114,...,0,0.0000,0.000000,0,0.0000,0.0,0,0.0000,0.00000,0
2,304.282774,2021-08-05 14:00:00,277,7698145312,3982474440,45948,82.869445,46939,0.505857,0.234593,...,0,0.0000,0.000000,0,0.0000,0.0,0,0.0000,0.00000,0
3,225.608906,2021-08-05 15:00:00,254,7075245729,3665086336,47237,64.957738,48827,0.221217,0.104981,...,0,0.0000,0.000000,0,0.0000,0.0,0,0.0000,0.00000,0
4,265.225009,2021-08-05 16:00:00,254,7107208896,3742906764,47726,80.700634,48625,0.391439,0.179006,...,0,0.0000,0.000000,0,0.0000,0.0,0,0.0000,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,35.935752,2022-08-05 08:00:00,261,7827886307,4088857563,8684,13.980088,50513,0.184175,0.069670,...,41,1369.6116,29971.993278,18,28941.1975,0.0,0,11397.1251,313807.36638,19
8757,27.898762,2022-08-05 09:00:00,262,7859908518,4130183041,8863,15.294690,54094,0.162364,0.110703,...,41,1369.6116,29971.993278,18,28941.1975,0.0,0,11397.1251,313807.36638,19
8758,23.378548,2022-08-05 10:00:00,257,7710993163,4051538305,9403,18.812820,53906,0.156575,0.103242,...,41,1369.6116,29971.993278,18,28941.1975,0.0,0,11397.1251,313807.36638,19
8759,56.087637,2022-08-05 11:00:00,260,7799821832,4441484858,11701,22.762670,60688,0.236753,0.119655,...,41,1369.6116,29971.993278,18,28941.1975,0.0,0,11397.1251,313807.36638,19


In [68]:
dataset.to_csv('./datasets/final_dataset.csv')